
## Turning OFF AQE amd DPP and AutoBroadcast

In [0]:
from pyspark.sql.functions import *


In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "false")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/rawdata/BigMart_Sales.csv")

df=df.limit(8)


**Preparing the Partition Data**

In [0]:
df.write.format("parquet").mode("append").partitionBy("Outlet_Type").option("path", "/FileStore/rawdata/dpp_partioned").save()



**non partitioned Data**

In [0]:
df.write.format("parquet").mode("append").option("path", "/FileStore/rawdata/dpp_nonpartioned").save()


**Dataframe**

In [0]:
df1=spark.read.format("parquet").option("path", "/FileStore/rawdata/dpp_partioned").load()
df2=spark.read.format("parquet").option("path", "/FileStore/rawdata/dpp_nonpartioned").load()

**Joins**

In [0]:
df_join = df1.join(
    df2.filter(col("Outlet_Type") == "Grocery Store"),
    df1["Item_Identifier"] == df2["Item_Identifier"],
    "inner"
)


In [0]:
df_join.display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Type,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,732.38,Grocery Store,FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38


click into it, you can see read 4 files in scan

**now use dpp**

In [0]:
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")


In [0]:
df.write.format("parquet").mode("append").partitionBy("Item_Identifier").option("path", "/FileStore/rawdata/dpp_partionednew").save()

In [0]:
df.write.format("parquet").mode("append").option("path", "/FileStore/rawdata/dpp_nonpartioned").save()

In [0]:
df1=spark.read.format("parquet").option("path", "/FileStore/rawdata/dpp_partionednew").load()
df2=spark.read.format("parquet").option("path", "/FileStore/rawdata/dpp_nonpartioned").load()

In [0]:
df_join = df1.join(
    df2.filter(col("Item_Identifier") == "FDA15"),
    df1["Item_Identifier"] == df2["Item_Identifier"],
    "inner"
)

df_join.display()

Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Identifier,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,FDA15,FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,FDA15,FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,FDA15,FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138


click into it, you can see read 1 files in scan

## partition on join key 

so ually date key is common choice